In [2]:
#SAVE_DIR = r'C:\Users\wyattluke.lowery\OneDrive - Texas A&M University\Research\GSP\Laplacian Library'
SAVE_DIR = r'C:\Users\wyatt\OneDrive - Texas A&M University\Research\GSP\Laplacian Library'
ROOT_DIR = r'C:\Users\wyatt\OneDrive - Texas A&M University\Research\GSP\Laplacian Library'

# Power system case file paths (Hawaii 37, EastWest ~80k, and Synthetic USA 82k)
DIR    = r"C:\Users\wyattluke.lowery\OneDrive - Texas A&M University\Research\Oscillations\Modal SGWT (Journal)"
TEXAS  = f"{DIR}\Case\ACTIVSg2000_250421.pwb", "TEXAS"
WECC   = f"{DIR}\Case\\240busWECC_2018_PSS.pwb", "WECC"
A70k   = r"C:\Users\wyatt\OneDrive - Texas A&M University\Research\Cases\Eastern 70k\ACTIVSg70k.pwb", "ACTIVSg70k"
DIR    = r"C:\Users\wyattluke.lowery\OneDrive - Texas A&M University\Research\Cases"
HAWAII = f"{DIR}\Hawaii\Hawaii40_20231026.pwb", "HAWAII"
EW     = f"{DIR}\East West 80k\Synthetic_East_West_US.pwb", "EASTWEST"
USA    = f"{DIR}\\USA\\SyntheticUSA.pwb", "USA"

from esapp import GridWorkBench

import numpy as np
import scipy.sparse as sp
import os
import scipy.io as sio

## Power World Conversion

### Laplacians

Validate Branches

In [3]:
def format_edges(B):
    
    # takes in sparse incidence matrix
    B = B.copy()

    B[B<0] = 0

    B.eliminate_zeros()
    B = B.tocoo()

    edges = np.stack([B.row, B.col], axis=1)

    # returns N x 2 array for source_idx, target_idx
    return edges

def write_edges_txt(filepath, edge_matrix, feature_names=None):
    # This function generates the edges.txt file

    data = np.array(edge_matrix)
    
    # Validate dimensions
    if data.ndim != 2 or data.shape[1] < 2:
        raise ValueError("Input matrix must be M x (2+K) with at least source and target columns.")

    num_features = data.shape[1] - 2

    # Handle feature names
    if feature_names is None:
        feature_names = [f"feature_{i+1}" for i in range(num_features)]
        
    if len(feature_names) != num_features:
        raise ValueError(f"Provided {len(feature_names)} names for {num_features} feature columns.")

    # Construct header
    header = ["source", "target"] + feature_names

    with open(filepath, 'w') as f:
        # Write Header
        f.write(" ".join(header) + "\n")
        
        # Write Rows
        for row in data:
            # Enforce integer formatting for source and target indices
            src, tgt = int(row[0]), int(row[1])
            
            # Format features (preserves float formatting from numpy)
            features = row[2:]
            
            if len(features) > 0:
                features_str = " ".join(map(str, features))
                f.write(f"{src} {tgt} {features_str}\n")
            else:
                f.write(f"{src} {tgt}\n")


In [5]:

def get_rng(x):
    xabs = np.abs(x).astype(float)
    return np.min(xabs), np.max(xabs)

for CASE, NAME in  [WECC,  HAWAII, TEXAS]: #EW, USA]: #A70k,

    print(f'Computing: {NAME}')

    # Load Case
    wb = GridWorkBench(CASE) 

    B = sp.csc_matrix(wb.network.incidence())

    E = format_edges(B)

    # Get Branch Weights
    W1  = wb.network.lengths() 
    W2 = wb.network.delay()
    W3 = np.abs(wb.network.ybranch())

    E = np.stack([
        E[:,0], E[:,1], W1, W2, W3
    ], axis=1)

    #write_edges_txt(f"{NAME}_edges.txt", E, ['length', 'delay', 'admittance'])

    # Print
    print(*get_rng(W1))

    # Close Case
    wb.esa.CloseCase()


# NOTE lengths are 'too large' and when 1/L^2 it looks like 0
# -----> 2000km seems like a lot, need to verify this, hm.


Computing: WECC
'open' took: 3.0967 sec
0.006923653008542061 2700.0000700354576
Computing: HAWAII
'open' took: 3.0958 sec
0.1 24.23119549177657
Computing: TEXAS
'open' took: 3.6628 sec
0.1 1924.3082598366072


In [ ]:
LAP_FOLDER = "DELAY"
COORD_FOLDER = "COORDS"

for CASE, NAME in  [A70k]#, WECC, HAWAII, EW, USA]:

    print(f'Computing: {NAME}')

    # Load Case
    wb = GridWorkBench(CASE) 

    LAP = wb.network.laplacian(BranchType.DELAY) 
    mat_path = f"{SAVE_DIR}\{LAP_FOLDER}\{NAME}_{LAP_FOLDER}.mat"
    sio.savemat(mat_path, {"A": LAP})

    # Make and Save Laplacian
    L1, L2 = wb.buscoords()
    out_dict = {
        "longitude": L1, "latitude": L2
    }
    mat_path = f"{SAVE_DIR}\{COORD_FOLDER}\{NAME}_{COORD_FOLDER}.mat"
    sio.savemat(mat_path, out_dict)

    # Close Case
    wb.io.esa.CloseCase()